In [3]:
import numpy as np
import pandas as pd

In [13]:
dataset = pd.read_csv('heart-disease-dataset.csv')
dataset.head()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [14]:
print(len(dataset))
print(dataset.columns)

1190
Index(['age', 'sex', 'chest pain type', 'resting bp s', 'cholesterol',
       'fasting blood sugar', 'resting ecg', 'max heart rate',
       'exercise angina', 'oldpeak', 'ST slope', 'target'],
      dtype='object')


In [19]:
df1 = dataset.drop('target',axis=1)
x = df1
y = dataset['target']

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
len(x_train)

952

In [32]:
from sklearn.linear_model import LogisticRegression
# Create and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)


LogisticRegression(max_iter=1000)

In [22]:

# Evaluate the model on the test set
accuracy = model.score(x_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.83


In [34]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=12)

cross_val_score(LogisticRegression(max_iter=2000), x, y, cv=cv)

array([0.85714286, 0.84033613, 0.81512605, 0.83193277, 0.79831933])

In [27]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier

In [57]:
def find_best_model_using_gridsearchcv(x, y):
    algos = {
        'logistic_regression': {
            'model': LogisticRegression(max_iter=2000),
            'params': {
                'C': [0.1, 1, 10],
                'penalty': ['l2']
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeClassifier(),
            'params': {
                'criterion': ['gini', 'entropy'],
                'max_depth': [10, 15, 17]
            }
        }
    }

    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=12)

    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False, error_score='raise')
        gs.fit(x, y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

In [58]:
find_best_model_using_gridsearchcv(x,y)

,model,best_score,best_params
0,logistic_regression,0.828571,"{'C': 1, 'penalty': 'l2'}"
1,lasso,0.179480,"{'alpha': 1, 'selection': 'cyclic'}"
2,decision_tree,0.879832,"{'criterion': 'entropy', 'max_depth': 17}"


In [37]:
cv = ShuffleSplit(n_splits=3, test_size=0.2, random_state=12)

cross_val_score(DecisionTreeClassifier(), x, y, cv=cv)

array([0.89915966, 0.87815126, 0.86134454])